# Dataset Validation for External/Custom Datasets

This notebook demonstrates how to validate external or custom datasets using `ydata-profiling`.

Contents:
- Requirements and installation
- Loading data from various sources (CSV, Excel, JSON, Parquet)
- Data preprocessing and transformation
- Generating comprehensive data profile reports
- Customising the validation report
- Comparing datasets
- Exporting reports for documentation

**Note:** This notebook is designed for datasets that come from external sources and may require preprocessing before validation.

## Requirements

- Python 3.9+
- `ydata-profiling` package (dataset profiling)
- `pandas` package (data manipulation)
- `ipywidgets` package (for interactive widgets)
- Optional: `openpyxl` (Excel files), `pyarrow` (Parquet files)

Quick installation:
```bash
pip install ydata-profiling pandas openpyxl pyarrow ipywidgets scikit-learn
```

In [ ]:
!pip install ydata-profiling pandas openpyxl pyarrow ipywidgets scikit-learn

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

print(f"pandas version: {pd.__version__}")

## Loading Data from Various Sources

Choose the appropriate method based on your data source. Uncomment and modify the relevant section.

### Example: Using a sample dataset for demonstration

In [ ]:
# Example: Create a sample dataset for demonstration
# Remove this cell when using your own data

from sklearn.datasets import fetch_california_housing

print("Loading sample dataset for demonstration...")
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['target'] = housing.target

# Add some synthetic issues for demonstration
rng = np.random.default_rng(42)
df.loc[rng.choice(df.index, 100, replace=False), 'AveRooms'] = np.nan  # Add missing values
df.loc[rng.choice(df.index, 50, replace=False), 'AveBedrms'] = np.nan
df = pd.concat([df, df.iloc[:20]])  # Add duplicates

print(f"Sample data loaded: {len(df)} rows, {len(df.columns)} columns")

## Data Preview and Initial Inspection

In [ ]:
# Preview the data
print("Data Preview (first 10 rows):")
df.head(10)

In [ ]:
# DataFrame info
print("DataFrame Info:")
df.info()

print("\nColumn Data Types:")
print(df.dtypes)

In [ ]:
# Basic statistics
print("Basic Statistics:")
df.describe()

In [ ]:
# Check for missing values
print("Missing Values Summary:")
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

## Data Preprocessing (Optional)

Apply transformations to prepare your data for profiling. These steps are optional and depend on your data quality requirements.

### Handling Data Types

In [ ]:
# Convert columns to appropriate data types
# Uncomment and adjust as needed

# Convert to datetime
# df['date_column'] = pd.to_datetime(df['date_column'], format='%Y-%m-%d')

# Convert to numeric
# df['numeric_column'] = pd.to_numeric(df['numeric_column'], errors='coerce')

# Convert to categorical
# df['category_column'] = df['category_column'].astype('category')

# Convert to string
# df['string_column'] = df['string_column'].astype(str)

print("Data types after conversion:")
print(df.dtypes)

### Handling Missing Values

In [ ]:
# Options for handling missing values (uncomment as needed)

# Option 1: Drop rows with any missing values
# df_clean = df.dropna()

# Option 2: Drop rows with missing values in specific columns
# df_clean = df.dropna(subset=['important_column1', 'important_column2'])

# Option 3: Fill missing values with a specific value
# df['column_name'] = df['column_name'].fillna(0)

# Option 4: Fill missing values with mean/median/mode
# df['numeric_column'] = df['numeric_column'].fillna(df['numeric_column'].mean())

# Option 5: Forward/backward fill
# df['column_name'] = df['column_name'].fillna(method='ffill')

print("Missing values handling: No changes applied (modify as needed)")

### Handling Duplicates

In [ ]:
# Check for duplicates
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

# Uncomment to remove duplicates
# df = df.drop_duplicates()
# print(f"After removing duplicates: {len(df)} rows")

### Column Renaming and Selection

In [ ]:
# Rename columns (uncomment and adjust as needed)
# df = df.rename(columns={
#     'old_name1': 'new_name1',
#     'old_name2': 'new_name2',
# })

# Select specific columns
# df = df[['column1', 'column2', 'column3']]

# Drop specific columns
# df = df.drop(columns=['unnecessary_column1', 'unnecessary_column2'])

print(f"Current columns: {list(df.columns)}")

## Generate Data Profile Report

Use `ydata-profiling` to generate a comprehensive data quality report.

In [ ]:
# Generate profile report
print("Generating data profile report...")
print("This may take a few minutes depending on dataset size.")

profile = ProfileReport(
    df,
    title="External Dataset Validation Report",
    explorative=True,
)

print("Profile report generated successfully.")

In [ ]:
# Display the report in the notebook
# Using to_notebook_iframe() for better compatibility
profile.to_notebook_iframe()

## Customising the Report

### Minimal Mode (for large datasets)

In [ ]:
# Minimal profile for large datasets
profile_minimal = ProfileReport(
    df,
    title="External Dataset - Minimal Report",
    minimal=True,  # Faster, less detailed
)

profile_minimal.to_notebook_iframe()

### Custom Configuration

In [ ]:
# Custom profile configuration
profile_custom = ProfileReport(
    df,
    title="External Dataset - Custom Report",
    # Correlation settings
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": True},
        "kendall": {"calculate": False},
        "phi_k": {"calculate": False},
    },
    # Missing values visualization
    missing_diagrams={
        "bar": True,
        "matrix": True,
        "heatmap": True,
    },
    # Sample size for interactions
    interactions={"continuous": False},
    # Duplicates detection
    duplicates={"head": 10},
)

profile_custom.to_notebook_iframe()

### Time Series Mode

If your dataset contains time series data, use the time series mode for specialised analysis.

To enable time series profiling:
1. Set your datetime column as the DataFrame index: `df_ts = df.set_index('datetime_column')`
2. Create the profile with `tsmode=True` and `sortby` parameter:
   ```python
   profile_ts = ProfileReport(
       df_ts,
       title="External Dataset - Time Series Report",
       tsmode=True,
       sortby="datetime_column",
   )
   profile_ts.to_notebook_iframe()
   ```

## Comparing Two Datasets

Compare two versions of a dataset (e.g., before/after cleaning, train/test split).

To compare datasets:
1. Create profiles for both datasets
2. Use the `compare()` method to generate a comparison report

```python
df_cleaned = df.dropna().drop_duplicates()

profile_original = ProfileReport(df, title="Original Dataset")
profile_cleaned = ProfileReport(df_cleaned, title="Cleaned Dataset")

comparison_report = profile_original.compare(profile_cleaned)
comparison_report.to_notebook_iframe()
```

## Export Reports

## Accessing Report Data Programmatically

In [ ]:
# Get the profile description as a dictionary
description = profile.get_description()

# Dataset overview
print("Dataset Overview:")
print(f"  Number of variables: {description.table['n_var']}")
print(f"  Number of observations: {description.table['n']}")
print(f"  Missing cells: {description.table['n_cells_missing']}")
print(f"  Missing cells (%): {description.table['p_cells_missing']:.2%}")
print(f"  Duplicate rows: {description.table['n_duplicates']}")
print(f"  Duplicate rows (%): {description.table['p_duplicates']:.2%}")

In [ ]:
# Variable-level statistics
print("\nVariable Types:")
for var_type, count in description.table['types'].items():
    print(f"  {var_type}: {count}")

# Alerts/warnings
print("\nAlerts:")
alerts = description.alerts
if alerts:
    for alert in alerts:
        print(f"  [{alert.alert_type.name}] {alert.column_name}: {alert.alert_type_name}")
else:
    print("  No alerts found.")

## Data Quality Checklist

Use this checklist to validate your dataset before using it for model training:

- [ ] **Completeness**: Missing values are within acceptable limits
- [ ] **Uniqueness**: Duplicate records have been addressed
- [ ] **Validity**: Data types are correct for each column
- [ ] **Accuracy**: Values are within expected ranges
- [ ] **Consistency**: Related columns have consistent values
- [ ] **Timeliness**: Data is current and relevant

In [ ]:
# Automated data quality checks
print("=" * 60)
print("DATA QUALITY SUMMARY")
print("=" * 60)


def get_quality_status(value, pass_threshold, warn_threshold):
    """Determine quality status based on thresholds."""
    if value < pass_threshold:
        return "PASS"
    if value < warn_threshold:
        return "WARN"
    return "FAIL"


def get_alert_status(alert_count, pass_threshold, warn_threshold):
    """Determine alert status based on count thresholds."""
    if alert_count <= pass_threshold:
        return "PASS"
    if alert_count < warn_threshold:
        return "WARN"
    return "FAIL"


# Completeness
missing_pct = description.table['p_cells_missing'] * 100
completeness_status = get_quality_status(missing_pct, pass_threshold=5, warn_threshold=20)
print(f"\nCompleteness: {completeness_status}")
print(f"  Missing cells: {missing_pct:.2f}%")

# Uniqueness
duplicate_pct = description.table['p_duplicates'] * 100
uniqueness_status = get_quality_status(duplicate_pct, pass_threshold=1, warn_threshold=5)
print(f"\nUniqueness: {uniqueness_status}")
print(f"  Duplicate rows: {duplicate_pct:.2f}%")

# Alerts summary
total_alerts = len(description.alerts)
alerts_status = get_alert_status(total_alerts, pass_threshold=0, warn_threshold=5)
print(f"\nAlerts: {alerts_status}")
print(f"  Total alerts: {total_alerts}")

print("\n" + "=" * 60)

## Tips and Troubleshooting

- **Large datasets**: Use `minimal=True` or sample your data:
  ```python
  df_sample = df.sample(n=10000, random_state=42)
  profile = ProfileReport(df_sample, minimal=True)
  ```

- **Memory issues**: Reduce dataset size or disable heavy computations:
  ```python
  profile = ProfileReport(df, interactions={"continuous": False}, correlations=None)
  ```

- **Encoding issues with CSV**: Try different encodings:
  ```python
  df = pd.read_csv(path, encoding='latin-1')  # or 'cp1252', 'iso-8859-1'
  ```

- **Date parsing issues**: Specify the format explicitly:
  ```python
  df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
  ```

- **High cardinality categorical columns**: Limit unique values shown:
  ```python
  profile = ProfileReport(df, vars={"cat": {"n_obs": 5}})
  ```

- **Widget rendering issues**: If `to_widgets()` fails with compatibility errors, use `to_notebook_iframe()` instead for reliable HTML rendering in the notebook.